# Descrição do Problema

Na investigação de fraudes em licitações, existem muitos desafios. Diferentes ações fraudulentas, pouco diferentes das não fraudulentas. Informações descentralizada entre os dados abertos governamentais e dados abertos do setor privado.  

# Objetivos

* investigar dados de licitações do GDF relacionados a prestação de serviço de TIC entre os anos de 2014 e 2020

* Identificar empresas inedonias que continuam disputando licitações
* Identificar empresas diferentes com mesmo corpo diretor
* Investigar contratos com dispensa de licitação realizados com essas empresas


# Recursos


- Trello
- Google Colab
- Python

In [1]:
# Bibliotecas Py

import pandas as pd
import requests
import json
import numpy as np
import itertools
import shutil
from google.colab import drive
from google.colab import files

# drive.mount('drive')

# Fontes de dados

Fontes de dados utilizadas:

* SIASG - Sistema Integrado de Administração de Serviços Gerais, Ministério da Economia
 
* CEIS - Cadastro Nacional de Empresas Inidôneas e Suspensas, Controladoria Gera da União - CGU

* CNEP - Cadastro Nacional de Empresas Punidas,

  - Portal de Compras Governamentais
    - [API](http://compras.dados.gov.br/)
    - [Dicionário de dados](http://compras.dados.gov.br/docs/lista-metodos-licitacoes.html)

  - Portal da Transparência
    - [API](http://api.portaldatransparencia.gov.br/)
    - [Documentação](#)
    - [Dicionário de dados](http://www.portaldatransparencia.gov.br/pagina-interna/603389-dicionario-de-dados-licitacoes)


* Projeto de compartilhamento de dados do ICIJ - [Datashare](https://datashare.icij.org/) 

#### ORGÃOS DO DF > UASG > LICITACOES > DESPESAS > FORNECEDORES

# Descricão dos dados - Transparência

### Órgãos do DF

### Despesas por órgão

In [ ]:
chave = {'chave-api-dados':'b07b2db3a9698e2a350ef9d36d0eb280'}

In [ ]:
json = []

for ano in range(2015, 2020):
  for i in orgaos:
    url= f'http://api.portaldatransparencia.gov.br/api-de-dados/despesas/por-orgao?ano={ano}&orgao={i}&pagina=1'
    # print(url)
    res = requests.get(url, verify=True,headers=chave).json()
    json = json + res
  
json = list(filter(None, json))
dfOrgaosDespesas = pd.DataFrame(json) 

In [ ]:
json[0]

In [ ]:
dfOrgaosDespesas.head()

# Descrição dos dados - compras.gov.br

ORGÃOS DO DF > UASG > LICITACOES > CONTRATOS > MODALIDADES 

PREGAO > RESULTADO > FORNECEDOR > ITENS > VALORES LANCES > PROPOSTAS > EVENTOS


In [3]:
url_compras_gov = 'http://compras.dados.gov.br/'
url_dados_fonte = 'dados_fonte/compras_gov/'

In [16]:
def getJsonComprasGov(**kwargs):
    print(url_compras_gov + kwargs.get('url'))

    nome = kwargs.get('nome')
    url = url_compras_gov + kwargs.get('url')

    res = requests.get(url, verify=True).json()
    print(f'----------------- status resposta: { res.stat ')
    print(f'----------------- total de { nome } econtrados: { res['count'] } ')
    # dfRes = pd.DataFrame(res)
    # dfRes.to_json(url_dados_fonte + nome + '.json')

    return res['_embedded'][nome]




SyntaxError: invalid syntax (2481025615.py, line 9)

In [13]:
def iterateJson(**kwargs):
    arr = []
    
    for i in kwargs.get('ids'):
        url = kwargs.get('url') + str(i),
        res = getJsonComprasGov(
            url = kwargs.get('url') + str(i),
            nome= kwargs.get('nome')
        )
        arr = arr + res

    return pd.DataFrame(arr)



    

### Órgãos do DF

#### metadados

- ativo	- Se o órgão está ativo.
- codigo - Código do órgão
- codigo_tipo_adm	- Código do tipo da administração do órgão
- codigo_tipo_esfera	- Tipo da esfera do órgão
- codigo_tipo_poder	- Código do tipo do poder do órgão
- nome	- Nome do órgão.

http://compras.dados.gov.br/licitacoes/doc/orgao/54000

#### execução

In [14]:
dfOrgaos = pd.DataFrame(
    getJsonComprasGov(
        url='licitacoes/v1/orgaos.json?nome=Distrito+federal',
        nome='Orgaos'
    )
)

dfOrgaos.head()

http://compras.dados.gov.br/licitacoes/v1/orgaos.json?nome=Distrito+federal


TypeError: can only concatenate str (not "int") to str

### UASGs relacionadas e Órgãos Superiores

#### metadados

UASG: Unidade Administrativa de Serviços Gerais do SIASG

- ativo -	Se a UASG está ativa.
- cep -	CEP da UASG.
- cnpj -	CNPJ da UASG
- id -	Identificador único da UASG no SICAF.
- id_municipio -	Identificador único do município da UASG.
- id_orgao -	Identificador único do órgão no SICAF.
- id_orgao_superior -	Identificador único do órgão Superior.
- nome -	Nome da UASG.
- total_fornecedores_cadastrados -	Quantidade total de fornecedores cadastrados na UASG.
- total_fornecedores_recadastrados -	Quantidade total de fornecedores recadastrados, no novo SICAF, na UASG.
- unidade_cadastradora -	Unidade cadastradora.

http://compras.dados.gov.br/docs/licitacoes/v1/uasgs.html

#### execucao

In [37]:

# dfUasg = iterateJson(
#     ids = dfOrgaos.codigo,
#     url = 'licitacoes/v1/uasgs.json?id_orgao=',
#     nome = 'uasgs'
# )

arrUasg = []
res = requests.get(url_compras_gov + 'licitacoes/v1/uasgs.json?id_orgao=16000', verify=True).json()
arrUasg = arrUasg + res['_embedded']['uasgs']
dfUasg = pd.DataFrame(arrUasg)

dfUasg.head()

,id,nome,id_orgao,id_orgao_superior,id_municipio,cnpj,cep,total_fornecedores_cadastrados,total_fornecedores_recadastrados,unidade_cadastradora,ativo,_links
0,100001,TRIBUNAL DE JUSTICA DO DISTRITO FEDERAL,16000,16000,97012.0,00531954000120,70094900,0,0,True,True,{'self': {'href': '/licitacoes/id/uasg/100001'...
1,100002,2A.CIRCUNSCRICAO JUDICIARIA,16000,16000,3018.0,00531954002840,69350000,0,0,False,True,{'self': {'href': '/licitacoes/id/uasg/100002'...
2,100003,1A.CIRCUNSCRICAO JUDICIARIA,16000,16000,NaN,None,68900000,0,0,False,False,{'self': {'href': '/licitacoes/id/uasg/100003'...
3,100004,2A.CIRCUNSCRICAO JUDICIARIA,16000,16000,NaN,None,68930000,0,0,False,False,{'self': {'href': '/licitacoes/id/uasg/100004'...
4,100005,3A.CIRCUNSCRICAO JUDICIARIA,16000,16000,NaN,None,68950000,0,0,False,False,{'self': {'href': '/licitacoes/id/uasg/100005'...


### Licitações por UASG

#### metadados

- codigo_do_item_no_catalogo -	Código do Item no Catálogo.
- data_abertura_proposta -	Data de abertura da proposta.
- data_entrega_edital -	Data de Entrega do Edital.
- data_entrega_proposta -	Data de entrega da proposta.
- data_publicacao -	Data da publicação da licitação.
- endereco_entrega_edital -	Endereço de Entrega do Edital.
- funcao_responsavel -	Função do Responsável pela Licitação.
- identificador -	Identificador da Licitação.
- informacoes_gerais -	Informações Gerais.
- modalidade -	Código da Modalidade da Licitação.
- nome_responsavel -	Nome do Responsável pela Licitação.
- numero_aviso -	Número do Aviso da Licitação.
- numero_item_licitacao -	Número Item Licitação.
- numero_itens -	Número de Itens.
- numero_processo -	Número do Processo.
- objeto -	Objeto da Licitação.
- situacao_aviso -	Situação do aviso.
- tipo_pregao -	Tipo do Pregão.
- tipo_recurso -	Tipo do Recurso.
- uasg -	Código da UASG.

http://compras.dados.gov.br/docs/licitacoes/v1/licitacoes.html 

#### execucao

In [ ]:
jsonLicitacoes = []
offset = 0
urlLicitacoes = url_compras_gov + 'licitacoes/v1/licitacoes.json'
# res = requests.get(url, verify=True, stream=True)
# print('------------------- total registros encontrados: ' + str(res['count']) )

for i in dfUasg.id:
    url = f'{urlLicitacoes}?uasg={i}'
    print(url)
    res = requests.get(url, verify=True, stream=True)
    res = res.json()
#     print(res)
    print('------------------- total registros encontrados: ' + str(res['count']) )
#     jsonLicitacoes = jsonLicitacoes + res['_embedded']['licitacoes']
    
#     while(res['count'] > 0):
#         offset = offset + 500
#         url = f'{url}&offset={offset}'
#         print(url)
#         res = requests.get(url, verify=True, stream=True).txt
#         print(res)
#         print('-------------------total registros encontrados offset: ' + str(res['count']))
#         jsonLicitacoes = jsonLicitacoes + res['_embedded']['licitacoes']
    
                

http://compras.dados.gov.br/licitacoes/v1/licitacoes.json?uasg=100001


In [28]:
dfLicitacoes.head()
#         print(res['_embedded']['uasg'])
#         jsonLicitacoes = jsonLicitacoes + res['_embedded']['licitacoes']

# dfLicitacoes = pd.DataFrame(jsonLicitacoes)

NameError: name 'dfLicitacoes' is not defined

In [ ]:

json['_embedded']['licitacoes']



In [ ]:
# http://compras.dados.gov.br/contratos/v1/contratos.html?uasg=20001&order_by=data_assinatura&order=desc

### Modalidades de licitações

In [ ]:
url = 'http://compras.dados.gov.br/licitacoes/v1/modalidades_licitacao.json'
json = requests.get(url, verify=True).json()

In [ ]:
dfModalidades = pd.DataFrame(json['_embedded']['ModalidadesLicitacao'])

In [ ]:
pd.set_option('display.max_colwidth', None)
dfModalidades[['codigo','descricao']]

### Grupos relacionados a TIC

In [ ]:
json = requests.get('http://compras.dados.gov.br/servicos/v1/grupos.json?descricao=TIC', verify=True).json()

obj = json['_embedded']['grupos']

df = pd.DataFrame(obj)
dfGrupos = df[['codigo','descricao']]
dfGrupos = dfGrupos[dfGrupos.codigo.isin([162,163,164,165,166,168,171,172,173,174,800])]

In [ ]:
dfGrupos

### Contratos vinculados as licitacões

### Despesas por Órgão - compras.dados.gov